In [ ]:
import os
import cv2
import sys
import json

import numpy as np
import skimage.io as io

from skimage.draw import polygon
from skimage import img_as_float

In [ ]:
root = '/content/drive/MyDrive/Cityscapes/final_train/'


In [ ]:
def get_all_labels(root):
    labels = []
    final_labels = {}

    img_folder =
    segmap_folder =

#     sub_path_train_img = os.path.join(root, img_folder, './train')
    sub_path_train_seg = os.path.join(root, segmap_folder, '/content/drive/MyDrive/Cityscapes/final_train/seg/train')

#     sub_path_val_img = os.path.join(root, img_folder, './val')
    #sub_path_val_seg = os.path.join(root, segmap_folder, './val')

    # extract training labels
    for folder in os.listdir(sub_path_train_seg):
#         curr_img_folder = os.path.join(sub_path_train_img, folder)
        curr_seg_folder = os.path.join(sub_path_train_seg, folder)
        for file in os.listdir(curr_seg_folder):
            if 'json' not in file:
                continue
            f = open(os.path.join(curr_seg_folder, file), 'r')
            data = json.loads(f.read())
            for obj in data['objects']:
                if obj['label'] not in labels:
                    labels.append(obj['label'])

    # extract validation labels
    #for folder in os.listdir(sub_path_val_seg):
#         curr_img_folder = os.path.join(sub_path_val_img, folder)
        #curr_seg_folder = os.path.join(sub_path_val_seg, folder)
        #for file in os.listdir(curr_seg_folder):
            #if 'json' not in file:
                #continue
            #f = open(os.path.join(curr_seg_folder, file), 'r')
            #data = json.loads(f.read())
            #for obj in data['objects']:
                #if obj['label'] not in labels:
                    #labels.append(obj['label'])

    for i in range(len(labels)):
        final_labels[labels[i]] = i
    return final_labels

In [ ]:
labels = get_all_labels(root)


ValueError: ignored

In [ ]:
labels

NameError: ignored

seg maps

In [ ]:
def create_segmentation_maps(root, labels):
    img_folder, segmap_folder = os.listdir(root)

    if not os.path.exists('./img'):
        os.makedirs('img')
    if not os.path.exists('./img/train'):
        os.makedirs('img/train')
    if not os.path.exists('./img/val'):
        os.makedirs('img/val')

    if not os.path.exists('./seg'):
        os.makedirs('seg')
    if not os.path.exists('./seg/train'):
        os.makedirs('seg/train')
    if not os.path.exists('./seg/val'):
        os.makedirs('seg/val')

    sub_path_train_img = os.path.join(root, img_folder, '/content/drive/MyDrive/Cityscapes/final_train/img/train')
    sub_path_train_seg = os.path.join(root, segmap_folder, '/content/drive/MyDrive/Cityscapes/final_train/seg/train')

    sub_path_val_img = os.path.join(root, img_folder, '/content/drive/MyDrive/Cityscapes/final_train/img/val')
    sub_path_val_seg = os.path.join(root, segmap_folder, '/content/drive/MyDrive/Cityscapes/final_train/seg/val')

    for folder in os.listdir(sub_path_train_seg):
        curr_img_folder = os.path.join(sub_path_train_img, folder)
        curr_seg_folder = os.path.join(sub_path_train_seg, folder)

        for file in os.listdir(curr_img_folder):
            id = file.split('_')[0]+'_'+file.split('_')[1]+'_'+file.split('_')[2]
            img = cv2.imread(os.path.join(curr_img_folder, file))

            f = open(os.path.join(curr_seg_folder, id+'_gtFine_polygons.json'), 'r')
            data = json.loads(f.read())
            seg_map = np.zeros((data['imgWidth'],data['imgHeight']))
            for obj in data['objects']:
                label = obj['label']
                poly = np.array(obj['polygon'])
                rr, cc = polygon(poly[:,0], poly[:,1], seg_map.shape)
                seg_map[rr,cc] = labels[label]

            cv2.imwrite('/content/drive/MyDrive/Cityscapes/final_train/img/train/'+id+'.png', img)
            cv2.imwrite('/content/drive/MyDrive/Cityscapes/final_train/seg/train/'+id+'.png', seg_map.T)

    for folder in os.listdir(sub_path_val_seg):
        curr_img_folder = os.path.join(sub_path_val_img, folder)
        curr_seg_folder = os.path.join(sub_path_val_seg, folder)

        for file in os.listdir(curr_img_folder):
            id = file.split('_')[0]+'_'+file.split('_')[1]+'_'+file.split('_')[2]
            img = cv2.imread(os.path.join(curr_img_folder, file))

            f = open(os.path.join(curr_seg_folder, id+'_gtFine_polygons.json'), 'r')
            data = json.loads(f.read())
            seg_map = np.zeros((data['imgWidth'],data['imgHeight']), np.uint8)
            for obj in data['objects']:
                label = obj['label']
                poly = np.array(obj['polygon'])
                rr, cc = polygon(poly[:,0], poly[:,1], seg_map.shape)
                seg_map[rr,cc] = labels[label]

            cv2.imwrite('/content/drive/MyDrive/Cityscapes/final_train/img/val/'+id+'.png', img)
            cv2.imwrite('/content/drive/MyDrive/Cityscapes/final_train/seg/val/'+id+'.png', seg_map.T)

In [ ]:
create_segmentation_maps(root, labels)


SubSets generator

In [ ]:
np.random.seed(42)

In [ ]:
train_subset_size, val_subset_size = 1965, 59

In [ ]:
train_img_path, train_seg_path = '/content/drive/MyDrive/Cityscapes/final_train/img/train/', '/content/drive/MyDrive/Cityscapes/final_train/seg/train/'
val_img_path, val_seg_path = '/content/drive/MyDrive/Cityscapes/final_train/img/val/', '/content/drive/MyDrive/Cityscapes/final_train/seg/val/'

In [ ]:
train_random_indices = np.random.permutation(len(os.listdir(train_img_path)))[:train_subset_size]
val_random_indices = np.random.permutation(len(os.listdir(val_img_path)))[:val_subset_size]

In [ ]:
if not os.path.exists('/content/drive/MyDrive/Cityscapes/final_train/img/subset_train'):
    os.makedirs('/content/drive/MyDrive/Cityscapes/final_train/img/subset_train')
if not os.path.exists('/content/drive/MyDrive/Cityscapes/final_train/img/subset_val'):
    os.makedirs('/content/drive/MyDrive/Cityscapes/final_train/img/subset_val')

if not os.path.exists('/content/drive/MyDrive/Cityscapes/final_train/seg/subset_train'):
    os.makedirs('/content/drive/MyDrive/Cityscapes/final_train/seg/subset_train')
if not os.path.exists('/content/drive/MyDrive/Cityscapes/final_train/seg/subset_val'):
    os.makedirs('/content/drive/MyDrive/Cityscapes/final_train/seg/subset_val')

In [ ]:
train_img_list, train_seg_list = os.listdir(train_img_path), os.listdir(train_seg_path)
for i in train_random_indices:
    img = cv2.imread(os.path.join(train_img_path, train_img_list[i]))
    seg = cv2.imread(os.path.join(train_seg_path, train_seg_list[i]))

    cv2.imwrite('/content/drive/MyDrive/Cityscapes/final_train/img/subset_train/'+str(i)+'.png', img)
    cv2.imwrite('/content/drive/MyDrive/Cityscapes/final_train/seg/subset_train/'+str(i)+'.png', seg)

error: ignored

In [ ]:
val_img_list, val_seg_list = os.listdir(val_img_path), os.listdir(val_seg_path)
for i in val_random_indices:
    img = cv2.imread(os.path.join(val_img_path, val_img_list[i]))
    seg = cv2.imread(os.path.join(val_seg_path, val_img_list[i]))

    cv2.imwrite('/content/drive/MyDrive/Cityscapes/final_train/img/subset_val/'+str(i)+'.png', img)
    cv2.imwrite('/content/drive/MyDrive/Cityscapes/final_train/seg/subset_val/'+str(i)+'.png', seg)



error: ignored